In [ ]:

import os
import cv2     as cv
import numpy   as np
from PIL import Image
from ultralytics import YOLO
import calibri as cal




class Data_Collector:

    def __init__(self,imgs_dir_path, cam_calibration_path):
        self.imgs_dir_path = imgs_dir_path
        data               = np.load(cam_calibration_path,allow_pickle=True)
        self.mtx           = data['mtx']
        self.dist          = data['dist']
    
    def take_image(self, img_name , cam_idx):

        abs_path = os.path.join(self.imgs_dir_path,img_name)

        cap = cv.VideoCapture(cam_idx)
        
        if not cap.isOpened():
            print("Cannot open camera")
            exit()

        ret, frame = cap.read()

        if not ret:
            print("Can't take an image. Exiting ....")
            cap.release()
            cv.destroyAllWindows()
            return None
        else:       
            img = np.asarray(frame)
            undistored_img = cal.undistort(img,self.mtx,self.dist)
            success = cv.imwrite(abs_path,undistored_img)
            
            if success:
                print(f"Image successfully saved to {abs_path}")
               
            else:
                print(f"An error occured. The image was not able to be captured.")
                             
        cap.release()
        cv.destroyAllWindows()
        return abs_path

    
    def get_img_count(self, filepath):
        
        img_dir = os.listdir(filepath)
        
        count = 0

        for file in img_dir:
            if file.endswith(".jpg") or file.endswith(".png") or file.endswith(".jpeg"):
                count+=1
        print(f"There is currently {count} images in your image directory.")
        return count + 1

    def find_camera_idx(self):
        
        for cam_idx in range(-1,10):

            cap = cv.VideoCapture(cam_idx)
            
            if cap.isOpened():
                print(f"Camera is open on idx {cam_idx}")


        




In [3]:
obj = Data_Collector("/home/mgarcia/Pictures/","./calibrationdata.npz")

In [4]:
obj.find_camera_idx()

Camera is open on idx -1
Camera is open on idx 2


[ WARN:0@31.519] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@31.523] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@31.525] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@31.526] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@31.531] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video3): can't open camera by index
[ERROR:0@31.534] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@31.535] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video4): can't open camera by index
[ERROR:0@31.538] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@31.538] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video5): can't open camera by index
[ERROR:0@31.543] global obsensor_uvc_stream_channel.cpp:158 getStreamChan

In [5]:
img_path = obj.take_image("img4.png",2)

Image successfully saved to /home/mgarcia/Pictures/img4.png


In [6]:
model = YOLO("../../object_detection_model/model/mg_model_6/best.pt")

In [7]:
results = model(source = img_path,conf = .6)



image 1/1 /home/mgarcia/Pictures/img4.png: 480x640 1 grey pallet, 1 white pallet, 500.8ms
Speed: 19.6ms preprocess, 500.8ms inference, 23.4ms postprocess per image at shape (1, 3, 480, 640)


In [8]:
for r in results:
    print(r.boxes)

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0., 1.])
conf: tensor([0.7560, 0.7332])
data: tensor([[328.7330, 148.2540, 498.9012, 255.5324,   0.7560,   0.0000],
        [ 78.1365, 128.0424, 218.0704, 237.4280,   0.7332,   1.0000]])
id: None
is_track: False
orig_shape: (480, 640)
shape: torch.Size([2, 6])
xywh: tensor([[413.8171, 201.8932, 170.1682, 107.2784],
        [148.1035, 182.7352, 139.9339, 109.3856]])
xywhn: tensor([[0.6466, 0.4206, 0.2659, 0.2235],
        [0.2314, 0.3807, 0.2186, 0.2279]])
xyxy: tensor([[328.7330, 148.2540, 498.9012, 255.5324],
        [ 78.1365, 128.0424, 218.0704, 237.4280]])
xyxyn: tensor([[0.5136, 0.3089, 0.7795, 0.5324],
        [0.1221, 0.2668, 0.3407, 0.4946]])
